In [1]:
import itertools
import pandas as pd
import numpy as np
import string
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Imputer
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, confusion_matrix

In [2]:
df = pd.read_csv(r'train.csv', 'train', delimiter=',')
df = df[['PassengerId', 'Pclass', 'Sex', 'Age',  'Embarked', 'Survived']] #'SibSp', 'Parch',
df.head()

,PassengerId,Pclass,Sex,Age,Embarked,Survived
0,1,3,male,22.0,S,0
1,2,1,female,38.0,C,1
2,3,3,female,26.0,S,1
3,4,1,female,35.0,S,1
4,5,3,male,35.0,S,0


In [3]:
Embarked_LE = preprocessing.LabelEncoder()
df['Embarked'] = df['Embarked'].astype(str)
df['Embarked'] = Embarked_LE.fit_transform(df['Embarked'])


Sex_LE = preprocessing.LabelEncoder()
df['Sex'] = df['Sex'].astype(str)
df['Sex'] = Sex_LE.fit_transform(df['Sex'])

In [4]:
X = df[['PassengerId', 'Pclass', 'Sex', 'Age', 'Embarked']] #'SibSp', 'Parch',
y = df[['Survived']]

In [21]:
onehotencoder = preprocessing.OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

In [25]:
print(X[0])

[ 0.  0.  1.  1.  1. 22.  2.]


In [5]:
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
X = imputer.fit_transform(X)
#X.head()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(X_train)

[[332.    1.    1.   45.5   2. ]
 [734.    2.    1.   23.    2. ]
 [383.    3.    1.   32.    2. ]
 ...
 [861.    3.    1.   41.    2. ]
 [436.    1.    0.   14.    2. ]
 [103.    1.    1.   21.    2. ]]


In [7]:
from sklearn.svm import SVC
model_SVM = SVC(kernel = 'linear', random_state = 0)
model_SVM.fit(X, y)

C:\Users\Sribalaji\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
y_pred = model_SVM.predict(X_test)

In [9]:
def ModelScore(y_test, y_pred):
    print("Training Set Score")
    print("Accuracy - "+str(accuracy_score(y_test, y_pred)))
    print("Precision Score - "+str(precision_score(y_test, y_pred)))
    print("f1 Score - "+str(f1_score(y_test, y_pred)))
    print("Recall Score - "+str(recall_score(y_test, y_pred)))
          
ModelScore(y_test, y_pred)

Training Set Score
Accuracy - 0.7821229050279329
Precision Score - 0.7536231884057971
f1 Score - 0.7272727272727273
Recall Score - 0.7027027027027027


In [10]:
test_df = pd.read_csv("test.csv")

In [11]:
test_df['Embarked'] = test_df['Embarked'].astype(str)
test_df['Embarked'] = Embarked_LE.fit_transform(test_df['Embarked'])

test_df['Sex'] = test_df['Sex'].astype(str)
test_df['Sex'] = Sex_LE.fit_transform(test_df['Sex'])

In [12]:
sub_X = test_df[['PassengerId', 'Pclass', 'Sex', 'Age',  'Embarked']] #'SibSp', 'Parch',

In [13]:
sub_X = imputer.transform(sub_X)

In [14]:
sub_y = model_SVM.predict(sub_X)

In [15]:
output = pd.DataFrame({'PassengerId':test_df['PassengerId'],
                            'Survived':sub_y})
output.to_csv('submission_v2.csv',index=False)

In [26]:
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 4, init = 'uniform', activation = 'tanh', input_dim = 7))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 4, init = 'uniform', activation = 'tanh'))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X, y, batch_size = 10, nb_epoch = 100)

C:\Users\Sribalaji\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", input_dim=7, units=4, kernel_initializer="uniform")`
  if __name__ == '__main__':
C:\Users\Sribalaji\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=4, kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\Users\Sribalaji\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app
C:\Users\Sribalaji\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
891/891 [==============================] - 1s 854us/step - loss: 0.6887 - acc: 0.6083
Epoch 2/100
891/891 [==============================] - 0s 191us/step - loss: 0.6767 - acc: 0.6162
Epoch 3/100
891/891 [==============================] - 0s 232us/step - loss: 0.6689 - acc: 0.6162
Epoch 4/100
891/891 [==============================] - 0s 185us/step - loss: 0.6665 - acc: 0.6162
Epoch 5/100
891/891 [==============================] - 0s 176us/step - loss: 0.6658 - acc: 0.6162
Epoch 6/100
891/891 [==============================] - 0s 160us/step - loss: 0.6654 - acc: 0.6184
Epoch 7/100
891/891 [==============================] - 0s 205us/step - loss: 0.6650 - acc: 0.6173
Epoch 8/100
891/891 [==============================] - 0s 181us/step - loss: 0.6651 - acc: 0.6162
Epoch 9/100
891/891 [==============================] - 0s 161us/step - loss: 0.6672 - acc: 0.6184
Epoch 10/100
891/891 [==============================] - 0s 171us/step - loss: 0.6646 - acc: 0.6184
Epoch 11/100
891/89

891/891 [==============================] - 0s 233us/step - loss: 0.4708 - acc: 0.8058
Epoch 84/100
891/891 [==============================] - 0s 160us/step - loss: 0.4694 - acc: 0.8013
Epoch 85/100
891/891 [==============================] - 0s 190us/step - loss: 0.4544 - acc: 0.8070
Epoch 86/100
891/891 [==============================] - 0s 228us/step - loss: 0.4540 - acc: 0.7946
Epoch 87/100
891/891 [==============================] - 0s 190us/step - loss: 0.4601 - acc: 0.7924
Epoch 88/100
891/891 [==============================] - 0s 171us/step - loss: 0.4573 - acc: 0.7901
Epoch 89/100
891/891 [==============================] - 0s 160us/step - loss: 0.4654 - acc: 0.7901
Epoch 90/100
891/891 [==============================] - 0s 155us/step - loss: 0.4632 - acc: 0.7924
Epoch 91/100
891/891 [==============================] - 0s 188us/step - loss: 0.4602 - acc: 0.7912
Epoch 92/100
891/891 [==============================] - 0s 189us/step - loss: 0.4498 - acc: 0.7980
Epoch 93/100
891/891 [=

In [27]:
y_pred_cc = classifier.predict(X_test)
ModelScore(y_test, y_pred_cc)

ValueError: Error when checking input: expected dense_19_input to have shape (7,) but got array with shape (5,)